In [1]:
import requests as rq
import time
import random
import lxml
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

In [2]:
response = rq.get('http://www.hogwartsishere.com/library/book/7391/')
doc = BeautifulSoup(response.text)


In [3]:
episodes = []
results = doc.find_all('li', class_="hover-white")
for result in results:
    episode = result.find('a')['href']
    episodes.append(episode)
episodes

['/library/book/7391/chapter/1/',
 '/library/book/7391/chapter/2/',
 '/library/book/7391/chapter/3/',
 '/library/book/7391/chapter/4/',
 '/library/book/7391/chapter/5/',
 '/library/book/7391/chapter/6/',
 '/library/book/7391/chapter/7/',
 '/library/book/7391/chapter/8/']

In [4]:
full_urls = []
for url in episodes:
    url = f"http://www.hogwartsishere.com/{url}"
    full_urls.append(url)
full_urls

['http://www.hogwartsishere.com//library/book/7391/chapter/1/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/2/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/3/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/4/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/5/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/6/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/7/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/8/']

In [9]:
urltest = "http://www.hogwartsishere.com//library/book/7391/chapter/1/"
response = rq.get(urltest)
doc = BeautifulSoup(response.content)
line_text = []
scripts = doc.find('div', class_="font-size-16 roboto")

print(scripts)

#for script in scripts:
#    line_text.append(script.contents)
#    print(line_text)

<div class="font-size-16 roboto">
                            	Scene:<br/><br/>A neighbourhood on a street called Privet Drive. An owl, sitting on the street sign flies off to reveal a mysterious appearing old man walking through a forest near the street. He stops at the start of the street and takes out a mechanical device and zaps all the light out of the lampposts. He puts away the device and a cat meows. The man, ALBUS DUMBLEDORE, looks down at the cat, which is a tabby and is sitting on a brick ledge.<br/><br/> <br/><br/>Dumbledore: I should have known that you would be here...Professor McGonagall.<br/><br/> <br/><br/>The cat meows, sniffs out and the camera pans back to a wall. The cats shadow is seen progressing into a human. There are footsteps and MINERVA MCGONAGALL is revealed.<br/><br/> <br/><br/>McGonagall: Good evening, Professor Dumbledore. Are the rumours true, Albus?<br/><br/> <br/><br/>Dumbledore: I'm afraid so, Professor. The good, and the bad.<br/><br/>McGonagall: An

In [15]:
parsed = re.split(r'(<br/>\w.*?<br/>)', str(scripts))
parsed

['<div class="font-size-16 roboto">\n                            \tScene:<br/>',
 '<br/>A neighbourhood on a street called Privet Drive. An owl, sitting on the street sign flies off to reveal a mysterious appearing old man walking through a forest near the street. He stops at the start of the street and takes out a mechanical device and zaps all the light out of the lampposts. He puts away the device and a cat meows. The man, ALBUS DUMBLEDORE, looks down at the cat, which is a tabby and is sitting on a brick ledge.<br/>',
 '<br/> <br/>',
 '<br/>Dumbledore: I should have known that you would be here...Professor McGonagall.<br/>',
 '<br/> <br/>',
 '<br/>The cat meows, sniffs out and the camera pans back to a wall. The cats shadow is seen progressing into a human. There are footsteps and MINERVA MCGONAGALL is revealed.<br/>',
 '<br/> <br/>',
 '<br/>McGonagall: Good evening, Professor Dumbledore. Are the rumours true, Albus?<br/>',
 '<br/> <br/>',
 "<br/>Dumbledore: I'm afraid so, Professo

In [27]:
clean_lines = []
for line in parsed:
    a = re.sub('(<br/>)', '', str(line))
    b = re.sub('(\n)', '', a)
    c = re.sub('(<div\s.*?\\t)', '', b)
    d = re.sub('</div>', '', c)
    clean_lines.append(d)
print(clean_lines)

['Scene:', 'A neighbourhood on a street called Privet Drive. An owl, sitting on the street sign flies off to reveal a mysterious appearing old man walking through a forest near the street. He stops at the start of the street and takes out a mechanical device and zaps all the light out of the lampposts. He puts away the device and a cat meows. The man, ALBUS DUMBLEDORE, looks down at the cat, which is a tabby and is sitting on a brick ledge.', ' ', 'Dumbledore: I should have known that you would be here...Professor McGonagall.', ' ', 'The cat meows, sniffs out and the camera pans back to a wall. The cats shadow is seen progressing into a human. There are footsteps and MINERVA MCGONAGALL is revealed.', ' ', 'McGonagall: Good evening, Professor Dumbledore. Are the rumours true, Albus?', ' ', "Dumbledore: I'm afraid so, Professor. The good, and the bad.", '', 'McGonagall: And the boy?', '', 'Dumbledore: Hagrid is bringing him.', '', 'McGonagall: Do you think it wise to trust Hagrid with so

In [28]:
clean_lines_ok = []
for line in clean_lines:
    if str(line) != '':
        if str(line) != ' ':
            clean_lines_ok.append(line)
clean_lines_ok
                  

['Scene:',
 'A neighbourhood on a street called Privet Drive. An owl, sitting on the street sign flies off to reveal a mysterious appearing old man walking through a forest near the street. He stops at the start of the street and takes out a mechanical device and zaps all the light out of the lampposts. He puts away the device and a cat meows. The man, ALBUS DUMBLEDORE, looks down at the cat, which is a tabby and is sitting on a brick ledge.',
 'Dumbledore: I should have known that you would be here...Professor McGonagall.',
 'The cat meows, sniffs out and the camera pans back to a wall. The cats shadow is seen progressing into a human. There are footsteps and MINERVA MCGONAGALL is revealed.',
 'McGonagall: Good evening, Professor Dumbledore. Are the rumours true, Albus?',
 "Dumbledore: I'm afraid so, Professor. The good, and the bad.",
 'McGonagall: And the boy?',
 'Dumbledore: Hagrid is bringing him.',
 'McGonagall: Do you think it wise to trust Hagrid with something as important as 